In [ ]:
import pandas as pd
import numpy as np

temp = pd.read_csv('Data/cv-valid-train.csv')
texts = temp['text']

words_list = np.hstack([x.split(' ') for x in texts])
words_list_df = pd.Series(words_list)
temp = words_list_df.value_counts()

common_words = temp[:1000]
for ind in range(1000,len(temp)):
    num_freq = common_words[-1]
    if temp[ind] == num_freq:
        common_words = pd.concat([common_words,temp[ind:ind+1]])
    else: break

temp = None
texts = None
words_list = None


df1 = pd.read_csv('Data/Samples_1.csv')
df2 = pd.read_csv('Data/Samples_2.csv')
df3 = pd.read_csv('Data/Samples_3.csv')
df = pd.concat([df1,df2,df3])
df['words'] = df['words'].where(df['words'].isin(common_words.index),'[UNRECOGNIZED]')
print(len(df))

df1 = None
df2 = None
df3 = None

In [ ]:
from sklearn.preprocessing import LabelBinarizer
from keras import models
from keras import layers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.utils.class_weight import compute_class_weight
from keras import regularizers
from keras.callbacks import EarlyStopping, ModelCheckpoint



X = df.drop('words',axis=1)
y_raw = df['words']

lb = LabelBinarizer()
y = lb.fit_transform(y_raw)
print(y.shape)


X_train,X_val,y_train,y_val = train_test_split(X,y)
ss = StandardScaler()
X_train_scale = ss.fit_transform(X_train)
X_val_scale = ss.transform(X_val)


y_integers = np.argmax(y_train, axis=1)
class_weights = compute_class_weight('balanced', np.unique(y_integers), y_integers)
d_class_weights = dict(enumerate(class_weights))
d_class_weights[0]= d_class_weights[0]/1000

if len(d_class_weights)<y.shape[1]:
    init_len = len(d_class_weights)
    diff = y.shape[1]-init_len
    for i in range(diff):
        ind = init_len + i
        d_class_weights[ind] = 0

early_stopping = [EarlyStopping(monitor='val_loss', patience=10), 
                  ModelCheckpoint(filepath='best_model.h5', monitor='val_loss', save_best_only=True)]

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

def visualize_training_results(results):
    history = results.history
    plt.figure()
    plt.plot(history['val_loss'])
    plt.plot(history['loss'])
    plt.legend(['val_loss', 'loss'])
    plt.title('Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.show()
    
    plt.figure()
    plt.plot(history['val_accuracy'])
    plt.plot(history['accuracy'])
    plt.legend(['val_acc', 'accuracy'])
    plt.title('Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.show()

In [ ]:
model = models.Sequential()
model.add(layers.Dropout(0.1,input_shape=(3000,)))
model.add(layers.Dense(2000,activation='tanh',kernel_regularizer=regularizers.l2(0.5)))
model.add(layers.Dense(1009,activation='softmax'))
model.compile(loss='categorical_crossentropy',optimizer='sgd',metrics=['accuracy'])



results=model.fit(X_train_scale,y_train,epochs=100,batch_size=1000,validation_data=(X_val_scale,y_val),
                       callbacks=early_stopping,class_weight = d_class_weights,verbose=False)

results_train = model.evaluate(X_train_scale,y_train)
print('----------')
print(f'Training Loss: {results_train[0]:.3} \nTraining Accuracy: {results_train[1]:.3}')
results_val = model.evaluate(X_val_scale,y_val)
print('----------')
print(f'Validation Loss: {results_val[0]:.3} \nValidation Accuracy: {results_val[1]:.3}')

visualize_training_results(results)

In [ ]:
import pickle
with open('model.pickle','wb') as f:
    pickle.dump(model,f)